In [1]:
%reload_ext autoreload
%autoreload 2

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from collections import OrderedDict
import numpy as np
import re
import h5py
from functools import partial
from itertools import product

#from calibration import pulse_filter, parse_ids, DataProcessing
#from karabo_data import by_index

#from karabo_data.geometry2 import AGIPD_1MGeometry
import time
from calibration import Display


# Wokflow

* General set up (Tab 0):
   
   - Module Number: Specify channel/module numbers that you want to analyse. Must be comma separated.
   - Pulse indices: Pulses that have x-rays. You can provide a range (start:stop:step) or comma separated (1,3,4 etc) or combination of range and individual pulses (1:10:2, 11, 14, 16 etc)
   - Region of interest (ROI x) in x for each module: Only range based (start:stop)
   - Region of interest (ROI y) in y for each module: Only range based (start:stop)
   
* Dark Run (Tab 1):
    
    - Dark Run Folder: (str) /gpfs/exfel/exp/MID/201931/proposal/raw/run_number
    - Train ids: Train indices to take average over. Range based (start:stop). Deafult is all train (:)
    - Process Darak button: To evalute average. This is done in parallel over modules. Doesn't block
      further analysis. Once results (average image and histograms) are available it be displayed automatically.
    - Using Pulses (slider) and module numbers (dropbox) on top one can visulaize data for each pulses or modules.

* Data Visualization (Tab 2):


In [2]:
config = dict(
    dark_run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0504',
    run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0491',
)

In [3]:
d = Display(config=config)
d.control_panel()

In [ ]:
module_numbers = [15, 14]

In [ ]:
dark_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0504"

def f(x):
    return x

dark_average = partial(
    DataProcessing,
    path=dark_folder,
    pulse_ids="1:100:2",
    train_index=by_index[200:300],
    rois=by_index[..., :, :],
    operation=partial(np.mean, axis=0))


futures = OrderedDict()

with ProcessPoolExecutor(max_workers=len(module_numbers)) as executor:
    for mod_no in module_numbers:
        futures[mod_no] = executor.submit(dark_average, mod_no)


In [ ]:
print(futures)

dark_data = { module:result.result() for module, result in futures.items()}

print(dark_data[15].shape)

In [ ]:
with h5py.File("dark_run.h5", "w") as f:
    for modno, data in dark_data.items():
        g = f.create_group(f"entry_1/instrument/module_{modno}")
        g.create_dataset('data', data=dark_data[modno])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


for idx, modno in enumerate(module_numbers):
    fig, ax = plt.subplots(1,2, figsize=(15,4), tight_layout=True)
    ax[0].imshow(dark_data[modno][0, 0, ...])
    ax[1].hist(dark_data[modno][:, 0, ...].ravel(), bins=400,  edgecolor='w')
    ax[0].set_aspect('auto')

In [ ]:
flat_field_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0491"

dark_subtraction = partial(
    DataProcessing,
    path=flat_field_folder,
    pulse_ids="1:250:2",
    train_index=by_index[250:280],
    rois=by_index[..., :, :],
    operation=partial(f),
    dark_run=dark_data)

futures_dark_sub = OrderedDict()

with ProcessPoolExecutor(max_workers=len(module_numbers)) as executor:
    for mod_no in module_numbers:
        futures_dark_sub[mod_no] = executor.submit(dark_subtraction, mod_no)

In [ ]:
print(futures_dark_sub)
dark_subtracted_data = { module:result.result() for module, result in futures_dark_sub.items()}

dark_subtracted_data[15].shape

In [ ]:
with h5py.File("dark_subtracted.h5", "w") as f:
    for modno, data in dark_subtracted_data.items():
        g = f.create_group(f"entry_1/instrument/module_{modno}")
        g.create_dataset('data', data=dark_subtracted_data[modno])

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter, gaussian_filter1d


%matplotlib inline

for idx in range(99, 100):
    fig, ax = plt.subplots(1,2, figsize=(15,4), tight_layout=True)
    ax[0].imshow(dark_subtracted_data[15][0, idx, 0, ...] )
    counts, bins, _ = ax[1].hist(dark_subtracted_data[15][:, idx, 0 , ...].ravel(), bins=1000)
    ax[0].set_aspect('auto')
    bin_centers = (bins[1:] + bins[:-1])/2.0
    #filtered = savgol_filter(counts, 21, 2)
    filtered = gaussian_filter(counts, 1.5)
    peaks, _ = find_peaks(filtered, height=200)
    ax[1].plot(bin_centers[peaks], filtered[peaks], 'go')
    print(bin_centers[peaks], filtered[peaks])
    ax[1].plot(bin_centers, filtered, 'k-')

In [ ]:
hist, bin_edges = np.histogram(dark_subtracted_data[15][:, 95, 0 , ...].ravel(), bins=1000)
bin_centers = (bin_edges[1:] + bin_edges[:-1])/2.0

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Heatmap(
        z=dark_subtracted_data[15][0, 96, 0, ...],
        colorscale = 'Viridis',
        showscale=False), 
    row=1, col=1)

fig.add_trace(
    go.Bar(
        x=bin_centers, 
        y=hist), 
    row=1, col=2)



fig.update_layout(height=400, width=900, showlegend=False)

fig.show()

In [ ]:
smooth = gaussian_filter(hist, 1.5)
peaks, _ = find_peaks(smooth)

print(smooth[peaks])
print(bin_centers[peaks])


peaks_values = smooth[peaks]
peaks_positions = bin_centers[peaks]

ind = np.argpartition(peaks_values, -3)[-3:]
print("values ", peaks_values[ind])
print("values ", peaks_positions[ind])

In [ ]:
%matplotlib inline
import math

def g(x, A, mu, sigma):
    return A * np.exp(-(x-mu)**2 / (sigma))

from lmfit import Model

# model_1 = models.GaussianModel(prefix='m1_')
# model_2 = models.GaussianModel(prefix='m2_')
# model_3 = models.GaussianModel(prefix='m3_')

model_1 = Model(g, prefix='m1_')
model_2 = Model(g, prefix='m2_') 
model_3 = Model(g, prefix='m3_')

model = model_1 + model_2 + model_3

params_1 = model_1.make_params(A=7000, mu=-30, sigma=400)
params_2 = model_2.make_params(A=10000, mu=10, sigma=100)
params_1.update(params_2)
params_3 = model_3.make_params(A=7000, mu=50, sigma=100)
params_1.update(params_3)

print(params_1)
output = model.fit(hist, params_1, x=bin_centers)

plt.plot(bin_centers, hist)
plt.plot(bin_centers, output.best_fit, 'r-', label='best fit')
plt.plot(bin_centers, output.init_fit, 'k--', label='init fit')

In [ ]:
with h5py.File("dark_subtracted.h5", "r") as f:
    data = f["entry_1/instrument/module_14/data"][:]



In [ ]:
data.shape

In [ ]:
def GaussSum(x,*p):
    n = int(len(p)/3)
    A = p[:n]
    w = p[n:2*n]
    c = p[2*n:3*n]
    y = sum([ A[i]*np.exp(-(x-c[i])**2./(w[i])) for i in range(n)])
    return y


In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter, gaussian_filter1d

from scipy.optimize import curve_fit
%matplotlib inline


for idx in range(6, 8):
    params = []
    fig, ax = plt.subplots(1,2, figsize=(15,4), tight_layout=True)
    #ax[0].imshow(data[0, idx, 0, ...] )
    counts, bins, _ = ax[1].hist(data[:, idx, 0 , ...].ravel(), bins=1000)
    #ax[0].set_aspect('auto')
    bin_centers = (bins[1:] + bins[:-1])/2.0
    filtered = gaussian_filter(counts, 1.5)
    peaks, _ = find_peaks(filtered, height=2000, distance=10)
    
    
    ax[1].plot(bin_centers[peaks], filtered[peaks], 'go')
    print(bin_centers[peaks], filtered[peaks])
    
    
    sigma = np.full((len(bin_centers[peaks])), 1)
    print(sigma)
    params.extend(filtered[peaks])
    params.extend(sigma)
    params.extend(bin_centers[peaks])
    print(params)
    #gauss_data = np.array([GaussSum(x, *params) for x in bin_centers])
    popt, pcov = curve_fit(GaussSum, bin_centers, filtered, p0=params)
    gauss_data = np.array([GaussSum(x, *popt) for x in bin_centers])
    ax[1].plot(bin_centers, gauss_data, 'r--')
    #ax[1].plot(bin_centers, filtered, 'k-')